In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
# General tools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# For transformations and predictions
from sklearn.preprocessing import FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import  cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from scipy.optimize import curve_fit
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import pairwise_distances
from sklearn import metrics
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt 
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

# For the tree visualization
import pydot
from IPython.display import Image
#from sklearn.externals.six import StringIO
from io import StringIO
from sklearn import tree
from matplotlib import pyplot


# For scoring
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.metrics import mean_squared_error as mse


# For validation
from sklearn.model_selection import train_test_split as split, GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import  roc_auc_score

%matplotlib inline    
pd.options.mode.chained_assignment = None 

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print(os.listdir("../input")) 

* # First impression

About L&T Financial Services (LTFS):
Headquartered in Mumbai, LTFS is one of India’s most respected & leading NBFCs providing finance for two wheeler, farm equipment, housing, infra & microfinance. With a strong parentage & stable leadership, it also has a flourishing Mutual Fund & Wealth Advisory business under its broad umbrella.



In [ ]:
#Read Data
df=pd.read_csv('../input/lt-vehicle-loan-default-prediction/train.csv',  infer_datetime_format=True)
df_tst=pd.read_csv('../input/lt-vehicle-loan-default-prediction/test.csv',infer_datetime_format=True)
df.head()

In [ ]:
df_info=pd.read_csv('../input/lt-vehicle-loan-default-prediction/data_dictionary.csv')
df_info

We'll use only train data

In [ ]:
print(df.shape)
print(df_tst.shape)

In [ ]:
print(df.info())

In [ ]:
df.describe()

In [ ]:
df.nunique()

In [ ]:
df.loan_default.value_counts()


# Cleaning the data****

In [ ]:
#Drop duplicates
df.drop_duplicates()
print(df.shape)
# No duplicates in data

In [ ]:
df.isnull().sum()

In [ ]:
#df.isna().sum()

# EDA


UniqueID, supplier_id, Current_pincode_ID, Employee_code_ID will be removed as they are unnecessary & do not have any directly related predicting power. We will remove Date.of.Birth after extracting year part from the date and age of customer calculation.

In [ ]:
df=df.drop(['UniqueID','Current_pincode_ID', 'Employee_code_ID'],axis=1)

In [ ]:
#df_nan = df[df.isna().any(axis=1)]
#Checking for differences in loan default rates between different categories in Employment.Type, assuming that NaN values represent unemployed customers

df["Employment.Type"] = df["Employment.Type"].fillna('Unemlployed')


In [ ]:
#New sub-group, only default loans for future calaculations
df1=df.loc[df.loan_default==1]

In [ ]:
group_by_emptype = df1.groupby(['Employment.Type'])['loan_default'].count()/df.groupby(['Employment.Type'])['loan_default'].count()
group_by_emptype

In [ ]:
#There is no significant difference between the group so drop NaN values (new 'Unemployed')
#df.dropna(inplace=True)
df=df.loc[df['Employment.Type'] != 'Unemlployed']
print(df.shape)


# Removing outliers

In [ ]:
ax = sns.scatterplot(x='disbursed_amount', y='asset_cost',data=df);

In [ ]:
df=df.loc[df['disbursed_amount']<=110000]
df=df.loc[df['asset_cost']<=125000]
print(df.shape)

In [ ]:
plt.figure(figsize=(12,5))
ax=df.disbursed_amount.hist(bins=100);

In [ ]:
plt.figure(figsize=(12,5))
ax=df.asset_cost.hist(bins=100);

A loan to value (LTV) ratio compares the size of the loan you're requesting to take out compared to the appraised value of the item you want to buy.
Lenders and others use LTVs to determine how risky a loan is, for loan approval or denial, and to determine whether mortgage insurance is required. A higher LTV ratio suggests more risk because there's a higher chance of default.
Put another way: The LTV ratio tells you how much of a property you truly own compared to how much you owe. The ratio is used for several types of loans, including home and auto loans (both purchases and refinances).
To calculate an LTV ratio, divide the amount of the loan by the appraised value of the asset securing the loan.

In [ ]:
df=df.loc[df['ltv']>30]
df=df.loc[df['ltv']<90]
print(df.shape)

In [ ]:
x = sns.scatterplot(x='disbursed_amount', y='asset_cost',hue= 'ltv', data=df);

In [ ]:
plt.figure(figsize=(12,5))
ax=df.ltv.hist(bins=100);

In [ ]:
#df['real_ltv']=df['disbursed_amount']/df['asset_cost']*100
#plt.figure(figsize=(12,5))
#ax=df.real_ltv.hist(bins=100);

In [ ]:
features = ['branch_id', 'supplier_id','manufacturer_id','State_ID']
df[features].hist(figsize=(15, 10));

In [ ]:
#Loand default by branch, is significant? 
sns.set(rc={'figure.figsize':(20,6)})
ax = sns.countplot(x="branch_id", hue="loan_default", data=df)

In [ ]:
#Data analysis of branch_id column and bucketing
#df_by_branch = df.groupby(['branch_id'])['loan_default'].size().to_frame(name='branch_size').reset_index()
#branch_sorted=df_by_branch.sort_values(by=['branch_size'])
#bins_names= list(range(1, 11))
#branch_sorted['branch_bin'] = pd.cut(branch_sorted['branch_size'], bins=10, labels=bins_names)
#df = pd.merge(left=df, right=branch_sorted, how='left', left_on='branch_id', right_on='branch_id')
#df.head()

In [ ]:
#UPD
df1=df.loc[df.loan_default==1]

In [ ]:
#Data analysis of branch_id column
group_by_branch = (df1.groupby(['branch_id'])['loan_default'].count()/df.groupby(['branch_id'])['loan_default'].count()).to_frame(name='branch_risk').reset_index()
group_by_branch

In [ ]:
df = pd.merge(left=df, right=group_by_branch, how='left', left_on='branch_id', right_on='branch_id')


In [ ]:
group_by_branch['branch_risk'].plot.bar();

In [ ]:
#Data analysis of supplier_id column and bucketing
#df_by_supplier = df.groupby(['supplier_id'])['loan_default'].size().to_frame(name='supplier_size').reset_index()
#supplier_sorted=df_by_supplier.sort_values(by=['supplier_size'])
#sup_bins_names= list(range(1, 101))
#supplier_sorted['supplier_bin'] = pd.cut(supplier_sorted['supplier_size'], bins=100, labels=sup_bins_names)
#df = pd.merge(left=df, right=supplier_sorted, how='left', left_on='supplier_id', right_on='supplier_id')
#df.head()
#Update for new sub-group, only default loans for future calaculations
#df1=df.loc[df.loan_default==1]
#group_by_supbin = (df1.groupby(['supplier_bin'])['loan_default'].count()/df.groupby(['supplier_bin'])['loan_default'].count()).to_frame(name='supplier_risk').reset_index()
#group_by_branch.to_frame(name='branch_risk').reset_index()
#df = pd.merge(left=df, right=group_by_supbin, how='left', left_on='supplier_bin', right_on='supplier_bin')


In [ ]:
df['manufacturer_id'].value_counts()

In [ ]:
df=df.loc[df['manufacturer_id']!=67]

In [ ]:
#Update for sub-group
df1=df.loc[df.loan_default==1]

In [ ]:
#Data analysis of manufacturer_id column
group_by_manufacturer = (df1.groupby(['manufacturer_id'])['loan_default'].count()/df.groupby(['manufacturer_id'])
                         ['loan_default'].count()).to_frame(name='manufacturer_risk').reset_index()
df = pd.merge(left=df, right=group_by_manufacturer, how='left', left_on='manufacturer_id', right_on='manufacturer_id')
#df.head()

In [ ]:
group_by_manufacturer

In [ ]:
#group_by_manufacturer['manufacturer_risk'].plot.bar();

In [ ]:
#UPD
df1=df.loc[df.loan_default==1]

In [ ]:
#Data analysis of State_ID column
group_by_state = (df1.groupby(['State_ID'])['loan_default'].count()/df.groupby(['State_ID'])['loan_default'].count()).to_frame(name='state_risk').reset_index()
df = pd.merge(left=df, right=group_by_state, how='left', left_on='State_ID', right_on='State_ID')
#df.head()

In [ ]:
group_by_state['state_risk'].plot.bar();

In [ ]:
df=df.drop([ 'branch_id', 'supplier_id', 'State_ID', 'manufacturer_id' ],axis=1)


In [ ]:

from datetime import date

df['customer_age']=date.today().year-pd.DatetimeIndex(df['Date.of.Birth']).year
#df.head()

In [ ]:
df['customer_age'].hist(figsize=(5,5))

In [ ]:
df=df.loc[df['customer_age']>20]
print(df.shape)

In [ ]:
df['customer_age'].hist(figsize=(10,5));

In [ ]:
df=df.drop(['Date.of.Birth'],axis=1)

In [ ]:
#Data analysis of customer age column and bucketing
df=df.sort_values(by=['customer_age'])
sup_bins_names= list(range(1, 6))
df['cust_age_bin'] = pd.cut(df['customer_age'], bins=5, labels=sup_bins_names)
#df.head()

In [ ]:
#Update for sub-group
df1=df.loc[df.loan_default==1]

In [ ]:
#group_by_agebin = df1.groupby(['cust_age_bin'])['loan_default'].count()/df.groupby(['cust_age_bin'])['loan_default'].count()
#group_by_agebin.hist()
ax=df.groupby(['cust_age_bin','loan_default'])['disbursed_amount'].mean().unstack().plot(figsize=(14,6));


In [ ]:
df=df.drop([ 'customer_age' ],axis=1)


In [ ]:
df['cust_age_bin']=df['cust_age_bin'].astype(int)

In [ ]:
df=df.replace({'Employment.Type': {'Salaried': 1, 'Self employed': 0}})
df = df.rename({'Employment.Type': 'is_salaried'}, axis=1)

In [ ]:
df['is_salaried'].value_counts()

In [ ]:
#Only 2018
df=df.drop(['DisbursalDate'],axis=1)

In [ ]:
df['MobileNo_Avl_Flag'].value_counts()

In [ ]:
df=df.drop(['MobileNo_Avl_Flag'],axis=1)

In [ ]:
features = ['Aadhar_flag', 'PAN_flag', 'VoterID_flag', 'Driving_flag', 'Passport_flag']  
df[features].sum()


In [ ]:
df['shared_documents']=df['Aadhar_flag']+ df['PAN_flag']+df['VoterID_flag']+ df['Driving_flag']+ df['Passport_flag']

In [ ]:
df=df.drop(['Aadhar_flag', 'PAN_flag', 'VoterID_flag', 'Driving_flag', 'Passport_flag'], axis=1)

In [ ]:
df['shared_documents'].value_counts(normalize=True).to_frame()

In [ ]:
ax=df.groupby(['shared_documents','loan_default'])['disbursed_amount'].mean().unstack().plot(figsize=(14,6));


# ****Credit score 

Credit score is one of the foremost terms that consumers get acquainted with when they start learning the basics of personal financing and lending. Credit score is a 3-digit numeric that holds and depicts the creditworthiness of an individual. The credit score ranges between 300 – 900, wherein any score close to 900 is considered excellent by the lenders. An individual is able to get loan approval easily and at a lesser interest rates.
How is the CIBIL score calculated?
There are four key factors that impact your CIBIL score:
* Payment history: Making late payments or defaulting on your EMIs has a negative impact on your score.
* Credit mix: Having a balanced mix between secured loans and unsecured loans is likely to have a positive impact.
* Multiple enquiries: Too many loan enquiries may have a negative impact on your score as it indicates that your loan burden may go up in the future.
* High credit utilisation: A high credit utilisation limit indicates a rising debt burden over time and may negatively impact your score.

 


In [ ]:
df['PERFORM_CNS.SCORE.DESCRIPTION'].value_counts()

In [ ]:
df=df.loc[df['PERFORM_CNS.SCORE.DESCRIPTION']!='Not Scored: More than 50 active Accounts found']

In [ ]:
df=df.drop(['PERFORM_CNS.SCORE.DESCRIPTION'], axis=1)

In [ ]:
#Replace all not scored accounts by score 100
#df['PERFORM_CNS.SCORE'].loc[(df['PERFORM_CNS.SCORE'] < 300)] = 100

In [ ]:
#df = df.rename(columns={'PERFORM_CNS.SCORE.DESCRIPTION': 'score_desc'})
df = df.rename(columns={'PERFORM_CNS.SCORE': 'cns_score'})

In [ ]:
df['cns_score'].loc[df['cns_score']>300].hist();

In [ ]:
def cns_score(score):
    if score<=100:
        return 0
    elif (score>100) & (score<200):
        return 1
    elif (score>=200) & (score<300):
        return 2
    elif (score>=300) & (score<400):
        return 3
    elif (score>=400) & (score<500):
        return 4
    elif (score>=500) & (score<600):
        return 5
    elif (score>=600) & (score <700):
        return 6
    elif (score>=700) & (score <800):
        return 7
    elif (score>=800) & (score <900):
        return 8
    elif (score>=900) & (score <1000):
        return 9
    else:
        return 10

In [ ]:
df['cns_score']=df['cns_score'].map(lambda x:cns_score(x))
df['cns_score'].value_counts()

In [ ]:
#Two models?
df['cns_score'].hist();

In [ ]:
df['score_ind']=df['cns_score'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
df['score_ind'].value_counts(normalize=True).to_frame()

# Two Data Frames

In [ ]:
df_no_score=df.loc[df['score_ind']==0]

In [ ]:
df_scored=df.loc[df['score_ind']==1]

#  EDA for data without scoring

In [ ]:
df_no_score=df_no_score.drop(['cns_score','PRI.NO.OF.ACCTS','PRI.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS','PRI.CURRENT.BALANCE',
                     'PRI.SANCTIONED.AMOUNT','PRI.DISBURSED.AMOUNT','SEC.NO.OF.ACCTS','SEC.ACTIVE.ACCTS','SEC.OVERDUE.ACCTS','SEC.CURRENT.BALANCE','SEC.SANCTIONED.AMOUNT',
                     'SEC.DISBURSED.AMOUNT','PRIMARY.INSTAL.AMT','SEC.INSTAL.AMT','NEW.ACCTS.IN.LAST.SIX.MONTHS','DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS','AVERAGE.ACCT.AGE',
                     'CREDIT.HISTORY.LENGTH','NO.OF_INQUIRIES', 'score_ind'], axis=1)
#'PERFORM_CNS.SCORE','PERFORM_CNS.SCORE.DESCRIPTION'

In [ ]:
#df_no_score=df_no_score.drop('ltv',axis=1)
df_no_score.head()

In [ ]:
df_no_score.shape

In [ ]:
#df_scored.shape

In [ ]:
sns.pairplot(data=df_no_score[['disbursed_amount','asset_cost','ltv','manufacturer_risk','state_risk','branch_risk','cust_age_bin']]);

In [ ]:
plt.figure(figsize=(10,10))
sns.heatmap(df_no_score.corr(),annot=True,cmap='coolwarm')
plt.savefig('heatmap.png')

In [ ]:
#The correlation between branch_risk and state_risk is 0.76, we'll drop state_risk in the next scored df

# Back to the scored data 

In [ ]:
df_scored['year_acc'] = df_scored['AVERAGE.ACCT.AGE'].str.extract('(\d+)(?:yrs)')
df_scored['mon_acc'] = df_scored['AVERAGE.ACCT.AGE'].str.extract('(?:\s)(\d+)(?:mon)')
df_scored['avg_loan_age_month']=df_scored['year_acc'].astype(int)*12+df_scored['mon_acc'].astype(int)


In [ ]:
#df_scored.head()

In [ ]:
df_scored['year_his'] = df_scored['CREDIT.HISTORY.LENGTH'].str.extract('(\d+)(?:yrs)')
df_scored['mon_his'] = df_scored['CREDIT.HISTORY.LENGTH'].str.extract('(?:\s)(\d+)(?:mon)')
df_scored['credit_history_month']=df_scored['year_his'].astype(int)*12+df_scored['mon_his'].astype(int)
#df.head()

In [ ]:
df_scored=df_scored.drop(['AVERAGE.ACCT.AGE','CREDIT.HISTORY.LENGTH','year_acc','mon_acc','year_his', 'mon_his'], axis=1)

In [ ]:
#df['cns_score'].value_counts(normalize=True).to_frame()

In [ ]:
df_scored = df_scored.rename(columns={'PRI.NO.OF.ACCTS': 'primary_loans_taken', 'PRI.ACTIVE.ACCTS':'primary_active_loans', 
                        'PRI.OVERDUE.ACCTS':'primary_prev_defaults', 'PRI.CURRENT.BALANCE':'primary_loans_balance', 'PRI.SANCTIONED.AMOUNT':'primary_sanctioned_amt', 
                        'PRI.DISBURSED.AMOUNT':'primary_disbursed_amt', 'SEC.NO.OF.ACCTS': 'sec_loans_taken', 'SEC.ACTIVE.ACCTS':'sec_active_loans', 
                        'SEC.OVERDUE.ACCTS':'sec_prev_defaults', 'SEC.CURRENT.BALANCE':'sec_loans_balance', 'SEC.SANCTIONED.AMOUNT':'sec_sanctioned_amt',
                        'SEC.DISBURSED.AMOUNT':'sec_disbursed_amt','PRIMARY.INSTAL.AMT': 'primary_emi', 'SEC.INSTAL.AMT':'sec_emi', 'NEW.ACCTS.IN.LAST.SIX.MONTHS':'new_acc',
                        'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS':'new_loan_defaults', 'NO.OF_INQUIRIES':'enquries'})

In [ ]:
df_scored=df_scored.drop(['state_risk','score_ind'], axis=1)

In [ ]:
#df_scored.head()

In [ ]:
#df_scored.shape

In [ ]:
pd.crosstab(df_scored['loan_default'], df_scored['cns_score'], normalize='columns')

In [ ]:
ax=df_scored.groupby(['cns_score','loan_default'])['disbursed_amount'].count().unstack().plot(figsize=(14,6))

In [ ]:
#df['cns_score'].map(lambda x:cns_score(x)).value_counts()

In [ ]:
df_scored1=df_scored.loc[df['loan_default']==1]

In [ ]:
#Loans default percentage by CNS score
group_by_score = df_scored1.groupby(['cns_score'])['loan_default'].count()/df_scored.groupby(['cns_score'])['loan_default'].count()
group_by_score.fillna(0).plot(figsize=(10,5));


In [ ]:
#Removing outliers
df_scored=df_scored.loc[df_scored['primary_loans_taken']<14]
df_scored=df_scored.loc[df_scored['primary_loans_balance']<4000000]
df_scored=df_scored.loc[df_scored['primary_loans_balance']>=0]
df_scored=df_scored.loc[df_scored['primary_emi']<25000000]
print(df_scored.shape)

What is an EMI?
The Equated Monthly Instalment (or EMI) consists of the principal portion of the loan amount and the interest. Therefore, EMI = principal amount + interest paid on the Car Loan. The EMI, usually, remains fixed for the entire tenure of your loan, and it is to be repaid over the tenure of the loan on a monthly basis.

In [ ]:
ax = sns.scatterplot(x='primary_loans_balance', y='primary_emi',data=df_scored)

In [ ]:
df_scored=df_scored.loc[df_scored['primary_prev_defaults']<=8]
sns.catplot(x='primary_loans_taken', y='primary_prev_defaults', data=df_scored);


In [ ]:
pd.crosstab(df_scored['primary_loans_taken'], df_scored['primary_prev_defaults'], normalize='columns')

In [ ]:
f, axes = plt.subplots(1, 2,figsize=(15,10))
sns.boxplot(x=df_scored['primary_loans_taken'],y=df_scored['primary_loans_balance'], ax=axes[0])
sns.boxplot(x=df_scored['primary_prev_defaults'],y=df_scored['primary_loans_balance'], ax=axes[1])
sns.despine(left=True, bottom=True)
axes[0].set(xlabel='loans_taken', ylabel='loans_balance')
axes[0].yaxis.tick_left()
axes[1].yaxis.set_label_position("right")
axes[1].yaxis.tick_right()
axes[1].set(xlabel='prev_defaults', ylabel='loans_balance');

In [ ]:
sns.pairplot(data=df_scored[['primary_loans_taken', 'primary_active_loans','primary_prev_defaults','primary_loans_balance',
                             'primary_sanctioned_amt','primary_disbursed_amt', 'primary_emi']]);

In [ ]:
df_scored['primary_default_ratio']=df_scored['primary_prev_defaults']/df_scored['primary_loans_taken']
df_scored=df_scored.drop(['primary_loans_taken','primary_prev_defaults'], axis=1)
#df_scored.head()

In [ ]:
columns_to_show = [ 'primary_active_loans', 'primary_loans_balance', 'primary_sanctioned_amt', 'primary_disbursed_amt', 'primary_emi', 'new_acc','new_loan_defaults', 'enquries']
df_scored_pri=df_scored.loc[df_scored['primary_loans_balance']>0]
df_scored_pri.groupby(['loan_default'])[columns_to_show].agg([np.mean, np.max])

In [ ]:
#Columns primary_disbursed_amt and primary_sanctioned_amt almost the same, so we will leave only primary_disbursed_amt and drop primary_sanctioned_amt
df_scored=df_scored.drop(['primary_sanctioned_amt'], axis=1)

In [ ]:
#Secondary accounts are those which the customer act as a co-applicant or gaurantor
#Indication of the existence of a secondary account
df_scored['secondary_acc_ind']=df_scored['sec_loans_balance'].apply(lambda x: 1 if x > 0 else 0)

In [ ]:
df_scored['secondary_acc_ind'].value_counts()

In [ ]:
#Due to the fact that most rows don't have data about secondary accounts, next columns  will be droped,
df_scored=df_scored.drop(['sec_loans_taken', 'sec_active_loans', 'sec_prev_defaults', 'sec_loans_balance', 'sec_sanctioned_amt', 'sec_disbursed_amt','sec_emi'], axis=1)

In [ ]:
#Enquries done by the customer for loans
#"Multiple enquiries: Too many loan enquiries may have a negative impact on your score as it indicates that your loan burden may go up in the future."
df_scored['enquries'].value_counts(normalize=True).to_frame()

In [ ]:
df_scored=df_scored.drop(['enquries'], axis=1)

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(df_scored.corr(),annot=True,cmap='coolwarm')
plt.savefig('heatmap.png')

In [ ]:
df_scored=df_scored.drop(['asset_cost', 'new_acc','credit_history_month'], axis=1)

In [ ]:
df_scored.head()

In [ ]:
print(df_scored.shape)

In [ ]:

#scaler = StandardScaler()
#scaled_no_score = scaler.fit_transform(df_no_score)


# Splitting the data for train and test

In [ ]:
x_ns_train, x_ns_test = split(df_no_score, train_size=0.7, random_state=11742, stratify=df_no_score.loan_default)
xs_train, xs_test = split(df_scored, train_size=0.7, random_state=11742,stratify=df_scored.loan_default)

# Logistic Regression for the non-scored df

In [ ]:
df_no_score['loan_default'].value_counts(normalize=True)

In [ ]:
X=x_ns_train.drop('loan_default', axis=1)
y=x_ns_train['loan_default']

In [ ]:
from sklearn.preprocessing import scale
Xs = scale(X)

In [ ]:
mod_no_score=LogisticRegression(class_weight='balanced')

In [ ]:
x_ns_train_lr_score=(cross_val_score(mod_no_score, Xs,y, cv=5,scoring='roc_auc'))

In [ ]:
np.mean(x_ns_train_lr_score)

In [ ]:
X=x_ns_test.drop('loan_default', axis=1)
y=x_ns_test['loan_default']

In [ ]:
Xs = scale(X)

In [ ]:
mod_no_score.fit(X, y)

In [ ]:
roc_auc_score(y_true=y,
              y_score=mod_no_score.predict_proba(X)[:,0])

# Logistic Regression for the scored df

In [ ]:
#df_scored['disbursed_amount']=np.log1p(df_scored['disbursed_amount'])
#df_scored['primary_loans_balance']=np.log1p(df_scored['primary_loans_balance'])
#df_scored['primary_disbursed_amt']=np.log1p(df_scored['primary_disbursed_amt'])
#df_scored['primary_emi']=np.log1p(df_scored['primary_emi'])

In [ ]:
df_scored['loan_default'].value_counts(normalize=True)

In [ ]:
X=xs_train.drop('loan_default', axis=1)
y=xs_train['loan_default']

In [ ]:
Xs = scale(X)

In [ ]:
mod_score=LogisticRegression(class_weight='balanced')

In [ ]:
xs_train_lr_score=(cross_val_score(mod_score, Xs,y, cv=5,scoring='roc_auc'))

In [ ]:
np.mean(xs_train_lr_score)

In [ ]:
X=xs_test.drop('loan_default', axis=1)
y=xs_test['loan_default']

In [ ]:
Xs = scale(X)

In [ ]:
mod_score.fit(X, y)

In [ ]:
roc_auc_score(y_true=y,
              y_score=mod_score.predict_proba(X)[:,0])

# Decision Tree Classifier 

In [ ]:
dt_model=DecisionTreeClassifier()

In [ ]:
my_cv = StratifiedShuffleSplit(n_splits=10, train_size=0.7, test_size=0.3)

In [ ]:
my_param_grid = {'min_samples_leaf': [10,20, 40, 60],
                 'min_weight_fraction_leaf': [0.01, 0.02, 0.05],
                 'criterion': ['gini', 'entropy'], 
                 'min_impurity_decrease': [1e-5, 1e-6, 1e-7]}

# for non-scored df

In [ ]:
dt_model_no_s = GridSearchCV(estimator=dt_model, 
                           param_grid=my_param_grid, 
                           cv=my_cv,
                           scoring='roc_auc')

In [ ]:
X=x_ns_train.drop('loan_default', axis=1)
y=x_ns_train['loan_default']

In [ ]:
Xs = scale(X)

In [ ]:
dt_model_no_s.fit(Xs, y)

In [ ]:
dt_model_no_s.best_estimator_

In [ ]:
dt_model_no_score = DecisionTreeClassifier(criterion='entropy', min_impurity_decrease=1e-06,
                       min_samples_leaf=20, min_weight_fraction_leaf=0.01)

In [ ]:
xs_train_dtc_score=(cross_val_score(dt_model_score, X,y, cv=5,  scoring='roc_auc'))

In [ ]:
np.mean(xs_train_dtc_score)

In [ ]:
X=x_ns_test.drop('loan_default', axis=1)
y=x_ns_test['loan_default']

In [ ]:
Xs = scale(X)

In [ ]:
dt_model_no_score.fit(Xs, y)

In [ ]:
roc_auc_score(y_true=y,
              y_score=dt_model_no_score.predict_proba(X)[:,0])

# for scored df

In [ ]:
dt_model_gs = GridSearchCV(estimator=dt_model, 
                           param_grid=my_param_grid, 
                           cv=my_cv,
                           scoring='roc_auc')

In [ ]:
X=xs_train.drop('loan_default', axis=1)
y=xs_train['loan_default']

In [ ]:
Xs = scale(X)

In [ ]:
dt_model_gs.fit(Xs, y)

In [ ]:
dt_model_gs.best_estimator_

In [ ]:
dt_model_score = DecisionTreeClassifier(min_impurity_decrease=1e-06, min_samples_leaf=40,
                       min_weight_fraction_leaf=0.01)

In [ ]:
xs_train_dtc_score=(cross_val_score(dt_model_score, X,y, cv=5,  scoring='roc_auc'))

In [ ]:
np.mean(xs_train_dtc_score)

In [ ]:
X=xs_test.drop('loan_default', axis=1)
y=xs_test['loan_default']

In [ ]:
Xs = scale(X)

In [ ]:
dt_model_score.fit(Xs, y)

In [ ]:
roc_auc_score(y_true=y,
              y_score=dt_model_score.predict_proba(X)[:,0])